## 1. Baseline code provided by uni

### 1.1 Import modules

In [2]:
import pandas as pd
import logging
import json
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import make_pipeline
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_absolute_error

### 1.2 Baseline function to create [predictions](https://github.com/larshanen/MLChallenge/tree/main/notebooks/predicted.json)

In [2]:
def main():
    # Set the logging level to INFO and set loading message
    logging.getLogger().setLevel(logging.INFO)
    
    # Load train and test sets and change all NA values to empty values
    logging.info("Loading training/test data")
    train = pd.DataFrame.from_records(json.load(open('../data/train.json'))).fillna("")
    test = pd.DataFrame.from_records(json.load(open('../data/test.json'))).fillna("")
    
    # Split the train set into train (75%) and validation (25%) sets
    logging.info("Splitting validation")
    train, val = train_test_split(train, stratify=train['year'], random_state=123)
    
    # Store a featurizer to transform the 'title' column into a bag-of-words format
    featurizer = ColumnTransformer(
        transformers=[("title", CountVectorizer(), "title")], remainder='drop')
    
    # Make a pipeline for the featurizer combined with a dummy regressor, that simply predicts the overall trained mean of the target variable
    dummy = make_pipeline(featurizer, DummyRegressor(strategy='mean'))

    # Make a pipeline for the featurizer and a ridge model, that aims to minimize the sum of squares
    ridge = make_pipeline(featurizer, Ridge())
    
    # Drop target variable column and fit both models
    logging.info("Fitting models")
    dummy.fit(train.drop('year', axis=1), train['year'].values)
    ridge.fit(train.drop('year', axis=1), train['year'].values)
    
    # Calculate and report both MAE's
    logging.info("Evaluating on validation data")
    err = mean_absolute_error(val['year'].values, dummy.predict(val.drop('year', axis=1)))
    logging.info(f"Mean baseline MAE: {err}")
    err = mean_absolute_error(val['year'].values, ridge.predict(val.drop('year', axis=1)))
    logging.info(f"Ridge regress MAE: {err}")
    
    # Let the ridge model predict on test set
    logging.info(f"Predicting on test")
    pred = ridge.predict(test)
    test['year'] = pred
    
    # Write JSON prediction file
    logging.info("Writing prediction file")
    test.to_json("predicted.json", orient='records', indent=2)

In [4]:
main()

INFO:root:Loading training/test data
INFO:root:Splitting validation
INFO:root:Fitting models
INFO:root:Evaluating on validation data
INFO:root:Mean baseline MAE: 7.8054390754858805
INFO:root:Ridge regress MAE: 5.812345349001838
INFO:root:Predicting on test
INFO:root:Writing prediction file


## 2. Team code

Please follow the instructions beneath when writing or adjusting code:

In [12]:
# Describe every piece of code with comments
# Include your name in every header so we can report our individual contributions (this is mandatory)

### 2.1 Explore baseline performance (Lars)

In [8]:
# Set the logging level to INFO and set loading message
logging.getLogger().setLevel(logging.INFO)
    
# Load train and test sets and change all NA values to empty values
logging.info("Loading training/test data")
train = pd.DataFrame.from_records(json.load(open('../data/train.json'))).fillna("")
test = pd.DataFrame.from_records(json.load(open('../data/test.json'))).fillna("")
    
# Split the train set into train (75%) and validation (25%) sets
logging.info("Splitting validation")
train, val = train_test_split(train, stratify=train['year'], random_state=123)
    
# Store a featurizer to transform the 'title' column into a bag-of-words format
featurizer = ColumnTransformer(
transformers=[("title", CountVectorizer(), "title")], remainder='drop')
    
# Make a pipeline for the featurizer combined with a dummy regressor, that simply predicts the overall trained mean of the target variable
dummy = make_pipeline(featurizer, DummyRegressor(strategy='mean'))

# Make a pipeline for the featurizer and a ridge model, that aims to minimize the sum of squares
ridge = make_pipeline(featurizer, Ridge())
    
# Drop target variable column and fit both models
logging.info("Fitting models")
dummy.fit(train.drop('year', axis=1), train['year'].values)
ridge.fit(train.drop('year', axis=1), train['year'].values)
    
# Calculate and report both MAE's
logging.info("Evaluating on validation data")
err = mean_absolute_error(val['year'].values, dummy.predict(val.drop('year', axis=1)))
logging.info(f"Mean baseline MAE: {err}")
err = mean_absolute_error(val['year'].values, ridge.predict(val.drop('year', axis=1)))
logging.info(f"Ridge regress MAE: {err}")

INFO:root:Loading training/test data
INFO:root:Splitting validation
INFO:root:Fitting models
INFO:root:Evaluating on validation data
INFO:root:Mean baseline MAE: 7.8054390754858805
INFO:root:Ridge regress MAE: 5.812345349001838


In [9]:
# Check what validation set looks like
val

,ENTRYTYPE,title,editor,year,publisher,author,abstract
2603,inproceedings,Use of Heuristic Knowledge in Chinese Language...,,1984,Association for Computational Linguistics,"[Yang, Yiming, Nishida, Toyoaki, Doshita, Shuji]",
3258,inproceedings,Deciding the Twins Property for Weighted Tree ...,,2012,Association for Computational Linguistics,"[Büchse, Matthias, Fischer, Anja]",
48785,inproceedings,Large Margin Neural Language Model,,2018,Association for Computational Linguistics,"[Huang, Jiaji, Li, Yi, Ping, Wei, Huang, Liang]",We propose a large margin criterion for traini...
8822,inproceedings,Plot-guided Adversarial Example Construction f...,,2021,Association for Computational Linguistics,"[Ghazarian, Sarik, Liu, Zixi, S M, Akash, Weis...",With the recent advances of open-domain story ...
24495,inproceedings,Towards a terminological resource for biomedic...,,2006,European Language Resources Association (ELRA),"[Nenadic, Goran, Okazaki, Naoki, Ananiadou, So...",One of the main challenges in biomedical text ...
...,...,...,...,...,...,...,...
12457,inproceedings,Annotating Events in an Emotion Corpus,,2014,European Language Resources Association (ELRA),"[Lee, Sophia, Li, Shoushan, Huang, Chu-Ren]",This paper presents the development of a Chine...
29939,inproceedings,POSTECH Submission on Duolingo Shared Task,,2020,Association for Computational Linguistics,"[Park, Junsu, Kwon, Hongseok, Lee, Jong-Hyeok]","In this paper, we propose a transfer learning ..."
63102,inproceedings,"Annotation Trees: LDC's customizable, extensib...",,2012,European Language Resources Association (ELRA),"[Wright, Jonathan, Griffitt, Kira, Ellis, Joe,...","In recent months, LDC has developed a web-base..."
19430,inproceedings,"Long Nights, Rainy Days, and Misspent Youth: A...",,2015,Association for Computational Linguistics,"[Bracewell, David]",


In [34]:
# Check what array with predicted values looks like
pred_array = ridge.predict(val.drop('year', axis=1))
print(len(pred_array))
pred_array

16479


array([2005.16848241, 2016.72621127, 2013.17436633, ..., 2014.96570928,
       2014.87896051, 2020.60932351])

In [19]:
# Check what array with true values looks like
true_array = val['year'].values
print(len(true_array))
true_array

16479


array(['1984', '2012', '2018', ..., '2012', '2015', '2016'], dtype=object)

### 2.2 Build upon baseline code (Lars)

This paragraph build upon the previous baseline code. It entails the following adjustments/additions chronologically:

- [x] Removal of dummy regressor, since ridge works better from the very start;
- [x] 5-fold cross validation to reduce variability (Ridge regress MAE (5.773));
- [x] Try sklearn's other feature vectorizers (tf-idf (5.384), ...);
- [ ] Perform custom preprocessing, tokenizations within sklearn;
- [ ] Smooth sparse matrices (?);
- [ ] Tune hyperparameters of feature vectorizers (n-gram size);
- [ ] Test for or include other columns (abstract, authors (?));
- [ ] Try tasks other than regression, like lazy learning (kNN)(?);
- [ ] Try BERTopic modelling;
- [ ] 

In [4]:
# Import extra modules
from sklearn.model_selection import cross_val_score, KFold
from sklearn.feature_extraction.text import TfidfVectorizer

In [5]:
# Set the logging level to INFO and set loading message
logging.getLogger().setLevel(logging.INFO)
    
# Load train and test sets and change all NA values to empty values
logging.info("Loading training/test data")
train = pd.DataFrame.from_records(json.load(open('../data/train.json'))).fillna("")
test = pd.DataFrame.from_records(json.load(open('../data/test.json'))).fillna("")
    
# Split the train set into train (80%) and validation (20%) sets, 5-folds
logging.info("Splitting validation")
num_folds = 5
k_fold = KFold(n_splits=num_folds, shuffle=True, random_state=123)
    
# Store a featurizer to transform the 'title' column into a bag-of-words format
featurizer_1 = ColumnTransformer(
    transformers=[("title", CountVectorizer(), "title")], remainder='drop')
featurizer_2 = ColumnTransformer(
    transformers=[("title", TfidfVectorizer(), "title")], remainder='drop')
featurizers = [featurizer_1, featurizer_2]

for i, featurizer in enumerate(featurizers):
    # Make a pipeline for the featurizer and a ridge model, that aims to minimize the sum of squares
    ridge_cv = make_pipeline(featurizer, Ridge())
    
    # Drop target variable column and fit both models
    logging.info(f"Fitting model with featurizer {i+1}")
    ridge_cv.fit(train.drop('year', axis=1), train['year'].values)
    
    # Calculate and report both MAE's
    logging.info("Evaluating on validation data")
    ridge_cv_scores = cross_val_score(ridge_cv, train.drop('year', axis=1), train['year'].values, cv=k_fold, scoring='neg_mean_absolute_error')
    logging.info(f"Ridge regress MAE with featurizer {i+1} ({num_folds}-fold cross-validated): {-ridge_cv_scores.mean()}")

INFO:root:Loading training/test data
INFO:root:Splitting validation
INFO:root:Fitting model with featurizer 1
INFO:root:Evaluating on validation data
INFO:root:Ridge regress MAE with featurizer 1 (5-fold cross-validated): 5.773010450586702
INFO:root:Fitting model with featurizer 2
INFO:root:Evaluating on validation data
INFO:root:Ridge regress MAE with featurizer 2 (5-fold cross-validated): 5.384430333156983


In [15]:
tfidf_vectorizer = TfidfVectorizer()
tfidf_matrix = tfidf_vectorizer.fit_transform(val['title'])
feature_names1 = tfidf_vectorizer.get_feature_names_out()
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)
tfidf_df.head()

,00,000,001,0099,01,02,03,04,07,08,...,雜訊環境下應用線性估測編碼於特徵時序列之強健性語音辨識,雜訊環境與說話內容因素分析之強健性語音辨認,電腦輔助句子重組試題編製,電話查詢口語對話系統中語音辨識不確定性之處理,電話轉接對話模式與表達轉接要求句型的分析,非負矩陣分解法於語音調變頻譜強化之研究,面向中文口语理解的基于依赖引导的字特征槽填充模型,面向对话文本的实体关系抽取,面向机器阅读理解的高质量藏语数据集构建,領域相關詞彙極性分析及文件情緒分類之研究
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
count_vectorizer = CountVectorizer()
count_matrix = count_vectorizer.fit_transform(val['title'])
feature_names2 = count_vectorizer.get_feature_names_out()
count_df = pd.DataFrame(count_matrix.toarray(), columns=feature_names)
count_df.head()

,00,000,001,0099,01,02,03,04,07,08,...,雜訊環境下應用線性估測編碼於特徵時序列之強健性語音辨識,雜訊環境與說話內容因素分析之強健性語音辨認,電腦輔助句子重組試題編製,電話查詢口語對話系統中語音辨識不確定性之處理,電話轉接對話模式與表達轉接要求句型的分析,非負矩陣分解法於語音調變頻譜強化之研究,面向中文口语理解的基于依赖引导的字特征槽填充模型,面向对话文本的实体关系抽取,面向机器阅读理解的高质量藏语数据集构建,領域相關詞彙極性分析及文件情緒分類之研究
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
